<a href="https://colab.research.google.com/github/Musa-Ali-Kazmi/Hand-drawn-Sketch-recognition-using-CNN/blob/main/SketchDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!sudo apt-get install unrar

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
!unrar x /content/drive/MyDrive/DL-Assignment2/Train.rar /content/drive/MyDrive/DL-Assignment2


In [ ]:
!unrar x /content/drive/MyDrive/DL-Assignment2/Validation.rar /content/drive/MyDrive/DL-Assignment2

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL
import PIL.Image

In [11]:
# Load the CSV file containing image filenames and labels
train_df = pd.read_csv('/content/drive/MyDrive/DL-Assignment2/Train.csv')

# Define ImageDataGenerator for data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0,1]
    rotation_range=20,  # Random rotation up to 20 degrees
    width_shift_range=0.2,  # Random shift width up to 20% of the image width
    height_shift_range=0.2,  # Random shift height up to 20% of the image height
    shear_range=0.2,  # Shear intensity (shear angle in counter-clockwise direction in degrees)
    zoom_range=0.2,  # Random zoom up to 20%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Strategy for filling in newly created pixels
)

train_df = train_df.astype(str)  # Convert all values to strings

# Flow images from the "Train" folder using the ImageDataGenerator and dataframe
train_generator = datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='/content/drive/MyDrive/DL-Assignment2/Train',  # Path to the directory containing the images
    x_col="Image",  # Column containing the image filenames
    y_col="Label",  # Column containing the image labels
    target_size=(224, 224),  # Resize images to 224x224
    batch_size=32,  # Batch size
    class_mode='raw'  # Type of label encoding
  )

Found 0 validated image filenames.


/usr/local/lib/python3.10/dist-packages/keras/src/preprocessing/image.py:1137: UserWarning: Found 10000 invalid image filename(s) in x_col="Image". These filename(s) will be ignored.
  warnings.warn(


In [ ]:
# Load ResNet-50 pre-trained on ImageNet without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(250, activation='relu')(x)  # Final layer with 250 neurons
predictions = Dense(250, activation='softmax')(x)

In [ ]:
# Combine the base model and custom classification head
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()